# Multilabel BERT Experiments

In this notebook we do some first experiments with BERT: we finetune a BERT model+classifier on each of our datasets separately and compute the accuracy of the resulting classifier on the test data.

For these experiments we use the `pytorch_transformers` package. It contains a variety of neural network architectures for transfer learning and pretrained models, including BERT and XLNET.

Two different BERT models are relevant for our experiments: 

- BERT-base-uncased: a relatively small BERT model that should already give reasonable results,
- BERT-large-uncased: a larger model for real state-of-the-art results.

In [1]:
PREFIX = "title9_but" #"voting_so_automl"
BERT_MODEL = 'bert-base-uncased'
BATCH_SIZE = 16 if "base" in BERT_MODEL else 2
GRADIENT_ACCUMULATION_STEPS = 1 if "base" in BERT_MODEL else 8
MAX_SEQ_LENGTH = 100

## Data

We use the same data as for all our previous experiments. Here we load the training, development and test data for a particular prompt.

In [2]:
import sys
sys.path.append('../')

import ndjson
import glob
import numpy as np
from collections import Counter

from quillnlp.models.bert.preprocessing import preprocess, create_label_vocabulary

data_file = f"../data/interim/{PREFIX}_withprompt.ndjson"

with open(data_file) as i:
    data = ndjson.load(i)
    
for item in data:
    item["label"] = item["labels"]
    
label2idx = create_label_vocabulary(data)
idx2label = {v:k for k,v in label2idx.items()}
target_names = [idx2label[s] for s in range(len(idx2label))]

data_items = preprocess(data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
data_items = np.array(data_items)
    
labels = Counter()
for item in data:
    labels.update(item["label"])
print(labels)

I0404 18:53:29.534177 140043817379648 file_utils.py:41] PyTorch version 1.2.0+cu92 available.
I0404 18:53:30.516104 140043817379648 file_utils.py:57] TensorFlow version 2.1.0 available.
I0404 18:53:31.153071 140043817379648 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


Counter({'Miscellaneous': 124, 'Teams_reinstated': 116, 'Women_sued': 107, 'Cuts_to_WSD,': 92, 'EMU_viol_TitleIX': 56, 'Court_Decision': 49, 'Cuts_to_mens_sports': 48, 'Judge_reinstate': 45, 'Affected_More_Men': 42, 'Did_not_cut_t_and_s': 30, 'Fem_athletes_mad': 22, 'Kept_other_ones': 20, 'Limit_woms_opps': 17, 'EMU_cut_less_men': 17, 'EMU_compliant': 15, 'Cuts_to_WSD': 1})


Read synthetic data, when available.

In [3]:
from collections import defaultdict

synth_files = glob.glob(f"../data/interim/{PREFIX}_withprompt_*.ndjson")

synth_data = []
for synth_file in synth_files:
    with open(synth_file) as i:
        synth_data.extend(ndjson.load(i))

preprocessed_synth_data = preprocess(synth_data, BERT_MODEL, label2idx, MAX_SEQ_LENGTH)
        
synth_map = defaultdict(list)
for item, preprocessed_item in zip(synth_data, preprocessed_synth_data):
    synth_map[item["source_text"]].append(preprocessed_item)

synth_map = {}

I0404 18:53:31.873625 140043817379648 tokenization_utils.py:501] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /home/yves/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


## Model

We load the pretrained model and put it on a GPU if one is available. We also put the model in "training" mode, so that we can correctly update its internal parameters on the basis of our data sets.

In [4]:
def evaluate_output(all_correct, all_predicted):
    correct = 0
    at_least_one = 0
    fp, fn, tp, tn = 0, 0, 0, 0
    for c, p in zip(all_correct, all_predicted):
        if sum(c == p) == len(c):
            correct +=1

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                at_least_one += 1
                break

        for ci, pi in zip(c, p):
            if pi == 1 and ci == 1:
                tp += 1
                same = 1
            elif pi == 1 and ci == 0:
                fp += 1
            elif pi == 0 and ci == 1:
                fn += 1
            else:
                tn += 1
                same =1

    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    print("Data size:", len(all_predicted))
    print("P:", tp, "/", tp+fp, "=", precision)
    print("R:", tp, "/", tp+fn, "=", recall)
    print("F:", 2*precision*recall/(precision+recall))
    print("A:", correct/len(all_correct))
    print("AL1:", at_least_one/len(all_correct))

In [5]:
import torch
import random

from quillnlp.models.bert.train import train, evaluate
from quillnlp.models.bert.models import get_multilabel_bert_classifier

from quillnlp.models.bert.preprocessing import get_data_loader
from sklearn.model_selection import KFold

random.shuffle(data_items)

kf = KFold(n_splits=5, shuffle=True, random_state=1)
all_correct, all_predicted = [], []
all_test_data = []
for train_idx, test_idx in kf.split(data_items):

    train_and_dev_data = data_items[train_idx]
    cutoff = int(len(train_and_dev_data)/6*5)
    train_data = train_and_dev_data[:cutoff]
    dev_data = train_and_dev_data[cutoff:]
    test_data = data_items[test_idx]
    
    print("Train size:", len(train_data))
    
    synth_data = []
    for item in train_data:
        synth_data.extend(synth_map.get(item.text, []))
                
    train_dataloader = get_data_loader(np.concatenate((train_data, synth_data)), BATCH_SIZE)
    dev_dataloader = get_data_loader(dev_data, BATCH_SIZE)
    test_dataloader = get_data_loader(test_data, BATCH_SIZE, shuffle=False)

    print("Final train dataloader length:", len(train_dataloader))
    
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), device=device)
    output_model_file = train(model, train_dataloader, dev_dataloader, BATCH_SIZE, GRADIENT_ACCUMULATION_STEPS, 
                              device, num_train_epochs=100)
    
    print("Loading model from", output_model_file)
    device="cpu"

    model = get_multilabel_bert_classifier(BERT_MODEL, len(label2idx), model_file=output_model_file, device=device)
    model.eval()
    
    _, _, test_correct, test_predicted = evaluate(model, test_dataloader, device)
    evaluate_output(test_correct, test_predicted)
    all_correct.extend(test_correct)
    all_predicted.extend(test_predicted)
    all_test_data.extend(test_data)

Train size: 460
Final train dataloader length: 29


I0404 18:53:32.406511 140043817379648 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0404 18:53:32.408109 140043817379648 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Loss history: []
Dev loss: 0.5807238121827444


Epoch:   1%|          | 1/100 [00:06<11:20,  6.87s/it]


Loss history: [0.5807238121827444]
Dev loss: 0.4713802585999171


...




Epoch:  72%|███████▏  | 72/100 [08:08<03:03,  6.54s/it]


Loss history: [0.5703889032204946, 0.45168088873227435, 0.3713988810777664, 0.31752805908521015, 0.2832884540160497, 0.2619638790686925, 0.2481021930774053, 0.23104599366585413, 0.20977324495712915, 0.19478107243776321, 0.18068071703116098, 0.17129809161027273, 0.15991338839133581, 0.15267540762821832, 0.14439847196141878, 0.14095059037208557, 0.13442849616209665, 0.1295055498679479, 0.12575887019435564, 0.12197569881876309, 0.11888336762785912, 0.11620859305063884, 0.1141701266169548, 0.11486967901388805, 0.10919986541072528, 0.10814833516875903, 0.10605745017528534, 0.10507416725158691, 0.10250762601693471, 0.10089514156182607, 0.10192418346802394, 0.09975786631306012, 0.09757911786437035, 0.09584837158521016, 0.09579815591375034, 0.09720322241385777, 0.0943191647529602, 0.09252247959375381, 0.09215047086278598, 0.09151112847030163, 0.09033517663677533, 0.09209948405623436, 0.09305020049214363, 0.09060696139931679, 0.08979978784918785, 0.08798108187814553, 0.08955100364983082, 0.088


Loss history: [0.5703889032204946, 0.45168088873227435, 0.3713988810777664, 0.31752805908521015, 0.2832884540160497, 0.2619638790686925, 0.2481021930774053, 0.23104599366585413, 0.20977324495712915, 0.19478107243776321, 0.18068071703116098, 0.17129809161027273, 0.15991338839133581, 0.15267540762821832, 0.14439847196141878, 0.14095059037208557, 0.13442849616209665, 0.1295055498679479, 0.12575887019435564, 0.12197569881876309, 0.11888336762785912, 0.11620859305063884, 0.1141701266169548, 0.11486967901388805, 0.10919986541072528, 0.10814833516875903, 0.10605745017528534, 0.10507416725158691, 0.10250762601693471, 0.10089514156182607, 0.10192418346802394, 0.09975786631306012, 0.09757911786437035, 0.09584837158521016, 0.09579815591375034, 0.09720322241385777, 0.0943191647529602, 0.09252247959375381, 0.09215047086278598, 0.09151112847030163, 0.09033517663677533, 0.09209948405623436, 0.09305020049214363, 0.09060696139931679, 0.08979978784918785, 0.08798108187814553, 0.08955100364983082, 0.088

I0404 19:32:17.245449 140043817379648 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at /home/yves/.cache/torch/transformers/4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.8f56353af4a709bf5ff0fbc915d8f5b42bfff892cbb6ac98c3c45f481a03c685
I0404 19:32:17.247045 140043817379648 configuration_utils.py:292] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": null,
  "do_sample": false,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "layer_norm_eps": 1e-12,
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embedd


Data size: 138
P: 119 / 133 = 0.8947368421052632
R: 119 / 157 = 0.7579617834394905
F: 0.8206896551724138
A: 0.7246376811594203
AL1: 0.8043478260869565


## Evaluation

In [6]:
evaluate_output(all_correct, all_predicted)

Data size: 692
P: 600 / 676 = 0.8875739644970414
R: 600 / 801 = 0.7490636704119851
F: 0.8124576844955991
A: 0.703757225433526
AL1: 0.7890173410404624


In [7]:
scores = {}
for item, predicted, correct in zip(all_test_data, all_predicted, all_correct):
    correct_labels = [idx2label[i] for i, l in enumerate(correct) if l == 1]
    predicted_labels = [idx2label[i] for i, l in enumerate(predicted) if l == 1]
    print("{}#{}#{}".format(item.text, ";".join(correct_labels), ";".join(predicted_labels)))    
        
    for label in predicted_labels + correct_labels:
        if label not in scores:
            scores[label] = {"tp": 0, "fp": 0, "fn": 0, "support": 0}
    
    for label in predicted_labels:
        if label in correct_labels:
            scores[label]["tp"] += 1
        else:
            scores[label]["fp"] += 1

    for label in correct_labels:
        scores[label]["support"] += 1
        if label not in predicted_labels:
            scores[label]["fn"] += 1
            
for label in scores:
    lp = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fp"]) if scores[label]["tp"] + scores[label]["fp"] > 0 else 0
    lr = scores[label]["tp"] / (scores[label]["tp"] + scores[label]["fn"]) if scores[label]["tp"] + scores[label]["fn"] > 0 else 0
    lf = 2*lp*lr/(lp+lr) if lp+lr > 0 else 0
    
    print(label, lp, lr, lf, scores[label]["support"])

Eastern Michigan University cut women's tennis and softball, but Chretien and Mayerova sued the University for violating Title IX.#Women_sued#Women_sued
Eastern Michigan University cut women's tennis and softball, but this is illegal according to Title IX.#Miscellaneous#
Eastern Michigan University cut women's tennis and softball, but students claimed they had broken the law specifically tittle IX#Women_sued#Women_sued
Eastern Michigan University cut women's tennis and softball, but I find a school with aviation but no scholarships or scholarships but no major."#Miscellaneous#Miscellaneous
Eastern Michigan University cut women's tennis and softball, but also cut several men's sports programs.#Cuts_to_mens_sports#Cuts_to_mens_sports
Eastern Michigan University cut women's tennis and softball, but also cut men's wrestling, as well as men's swimming and diving.#Cuts_to_WSD,#Cuts_to_WSD,
Eastern Michigan University cut women's tennis and softball, but had to reinstate the teams due to Titl

In [8]:
print(len([p for p in all_predicted if sum(p) == 0]), "/", len(all_predicted))

84 / 692
